In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# API 키 정보 로드
load_dotenv()

# 프로젝트 이름을 입력합니다.
logging.langsmith("reranker_VectorStores")

In [ ]:
# 문서를 예쁘게 출력하기 위한 도우미 함수
def pretty_print_docs(docs):
    for doc in docs:
        print("palce", doc.metadata["place"])
        print("oneroom_half_year", doc.metadata["oneroom_half_year"])
        print("oneroom_year", doc.metadata["oneroom_year"])
        print("tworoom_half_year", doc.metadata["tworoom_half_year"])
        print("tworoom_year", doc.metadata["tworoom_year"])
        print("\n=====================================\n")

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
from langchain_chroma import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings

# 저장할 경로 지정
DB_PATH = "./chroma_db"

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# # 디스크에서 문서를 로드합니다.
persist_db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=embeddings,
    collection_name="rental_data_with_null",
)

In [ ]:
# 저장된 데이터 확인
persist_db.get()

In [ ]:
# retriever = persist_db.as_retriever(
#     search_kwargs={"k": 5}
# )

# pretty_print_docs(retriever.invoke("기숙사 자취방 추천해줘"))

In [ ]:
# self-query retriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

# 메타데이터 필드 정보 생성
metadata_field_info = [
    AttributeInfo(
        name="place",
        description="The location information of the rental room. One of ['정문', '중문', '후문', '기숙사', '농가마트', '교육문화회관']",
        type="string",
    ),
    AttributeInfo(
        name="oneroom_half_year",
        description="The half-year rent price for a one-room unit.",
        type="integer",
    ),
    AttributeInfo(
        name="oneroom_year",
        description="The annual rent price for a one-room unit.",
        type="integer",
    ),
    AttributeInfo(
        name="tworoom_half_year",
        description="The half-year rent price for a two-room unit.",
        type="integer",
    ),
    AttributeInfo(
        name="tworoom_year",
        description="The annual rent price for a two-room unit.",
        type="integer",
    ),
]

In [ ]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# 문서 내용 설명과 메타데이터 필드 정보를 사용하여 쿼리 생성기 프롬프트를 가져옵니다.
prompt = get_query_constructor_prompt(
    "Brief summary of a rental room",  # 문서 내용 설명
    metadata_field_info,  # 메타데이터 필드 정보
)

# StructuredQueryOutputParser 를 생성
output_parser = StructuredQueryOutputParser.from_components()

# query_constructor chain 을 생성
query_constructor = prompt | llm | output_parser

In [ ]:
from langchain.retrievers.self_query.chroma import ChromaTranslator

retriever = SelfQueryRetriever(
    query_constructor=query_constructor,  # 이전에 생성한 query_constructor chain 을 지정
    vectorstore=persist_db,  # 벡터 저장소를 지정
    structured_query_translator=ChromaTranslator(),  # 쿼리 변환기
    search_kwargs={"k": 10},  # 검색 옵션
)

In [ ]:
pretty_print_docs(retriever.invoke("년세 300~400 후문 자취방 추천해줘"))

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# 모델 초기화
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

# 상위 3개의 문서 선택
compressor = CrossEncoderReranker(model=model, top_n=3)

# 문서 압축 검색기 초기화
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

# 압축된 문서 검색
# compressed_docs = compression_retriever.invoke("중문 자취방 추천해줘")

# # 문서 출력
# pretty_print_docs(compressed_docs)

In [ ]:
# 압축된 문서 검색
compressed_docs = compression_retriever.invoke("원룸년세 300이하 정문 자취방 추천해줘")

# 문서 출력
pretty_print_docs(compressed_docs)